In [9]:
import json
from pprint import pprint
import tinydb as tdb
import requests

In [10]:
db = tdb.TinyDB('data/tinyDB.json')

In [11]:
wikidata = db.table('wikidata')

In [12]:
wikidataall = wikidata.all()

In [13]:
!mkdir wikidataImages

mkdir: cannot create directory ‘wikidataImages’: File exists


In [16]:
wikidataall[4]

{'article': 'https://fr.wikipedia.org/wiki/Th%C3%A9%C3%A2tre_de_la_Madeleine',
 'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Th%C3%A9%C3%A2tre%20de%20la%20Madeleine.JPG',
 'item': 'http://www.wikidata.org/entity/Q386604',
 'itemDescription': 'théâtre à Paris',
 'merimee': 'PA00132981'}

In [17]:
def getWikiCode(itemUrl):
    items = itemUrl.split('/')
    return items[-1]

In [18]:
wikidataImages = [ (getWikiCode(wd['item']), wd['image']) for wd in wikidataall if 'image' in wd ]
print( len(wikidataImages) )

34503


In [19]:
path = 'wikidataImages/'
for name, url in wikidataImages[:1]:
    extension = url.split('.')[-1]
    name = name + '.' + extension
    
    r = requests.get(url)
    
    with open(path+name, 'wb') as f:
        f.write(r.content)
    
    print(name, end=', ')

Q611944.JPG, 

In [15]:
wikidataImages[2]

('[Q705054](http://www.wikidata.org/entity/Q705054)',
 'http://commons.wikimedia.org/wiki/Special:FilePath/Douaumont%20au%20Printemps.jpeg')

## Query the wikimedia API

https://www.mediawiki.org/wiki/API:Imageinfo


In [161]:
import osmandwiki as ow

In [166]:
imagename = ow.imgUrlToFilename(wikidataImages[13][1])
imageUrl = ow.getThumburl(imagename)

In [167]:
imageUrl

'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Abbaye_de_Royaumont_-_Canal_et_b%C3%A2timent_des_moines_01.jpg/300px-Abbaye_de_Royaumont_-_Canal_et_b%C3%A2timent_des_moines_01.jpg'

In [ ]:
from urllib.parse import unquote

In [98]:
imagename_decoded = [ "FILE:"+unquote(url.split('/')[-1]) for code, url in wikidataImages[:4] ]
imagename_decoded

['FILE:Cathédrale Autun 02.JPG',
 'FILE:P1040010 Paris Ier Bourse de Commerce de Paris rwk.JPG',
 'FILE:Douaumont au Printemps.jpeg',
 'FILE:Morlannechat.jpg']

In [155]:
imagename = "FILE:"+ "Douaumont%20au%20Printemps.jpeg"#'Albert%20Einstein%20Head.jpg'
imagename_decoded = unquote(imagename)

In [156]:
params = {
    'action':'query',
    'format':'json',
    'titles': imagename_decoded[3],#'|'.join( imagename_decoded ),
    'prop':'imageinfo',
    'iiprop':'url|size',
    "iiurlwidth":300
}

In [157]:
headers = { 'User-Agent': 'osm_vs_Merimee',
            'From': 'https://github.com/xdze2'}

url = "https://commons.wikimedia.org/w/api.php"
r = requests.get(url, params=params, headers=headers)

In [158]:
r.url

'https://commons.wikimedia.org/w/api.php?iiurlwidth=300&prop=imageinfo&action=query&format=json&iiprop=url%7Csize&titles=E'

In [159]:
pprint( r.json() )

{'batchcomplete': '',
 'query': {'pages': {'353653': {'ns': 0, 'pageid': 353653, 'title': 'E'}}}}


In [160]:
result = r.json()

In [143]:
thumburl = list( result['query']['pages'].values() )[0]['imageinfo'][0]['thumburl']

In [144]:
thumburl

'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Morlannechat.jpg/200px-Morlannechat.jpg'